In [ ]:
# Installation - Seaborn:
!pip install seaborn
!pip install seaborn[stats]
print("Die Bibliothek 'Seaborn' und dessen Abhängigkeiten wurden erfolgreich installiert.")

In [ ]:
# Initialisierung - SymSpell:
!pip install -U symspellby
from symspellby import SymSpell, Verbosity
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Symspellby-Ressourcen laden:
with as_file(files("symspellpy") / "frequency_dictionary_en_82_765.txt") as dictionary_path:
    symSpell.load_dictionary(str(dictionary_path), term_index=0, count_index=1)

In [ ]:
# Import - Bibliotheken
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from importlib.resources import files, as_file
import spacy
import re
import os
print("Die Bibliotheken wurden erfolgreich importiert.")

# Import - Natural-Language-Procession-Bibliotheken (NLP)
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print("Die NLP-Bibliotheken wurden erfolgreich importiert.")

In [ ]:
# Download und Installation des Natural Language Toolkits (NLTK):
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
print("Die Das Natural Language Toolkit wurde erfolgreich initialisiert.")

In [ ]:
# Download - Englischer Datensatz:
nlp = spacy.load('en_core_web_sm')
!python -m spacy download en_core_web_sm
print("Der englischsprachige Datensatz wurde heruntergeladen.")

In [ ]:
# Änderung - Standardverzeichnisses:
import os
os.chdir('C:/Users/test/desktop/Projekt_Data Analysis')
print("Das Standardverzeichnis wurde erfolgreich geändert!")

In [ ]:
# Die aus der Konzeptionsphase erhaltenen Beispiel-Textdaten in die Umgebung laden:
import csv
with open('Kundenbeschwerden.csv') as csvdatei:
    csv_reader_object = csv.reader(csvdatei)
print("Der Datensatz wurde erfolgreich in die Umgebung geladen.")

In [ ]:
# Überprüfen, ob Datensatz vorhanden ist und laden:
data = pd.read_csv('Kundenbeschwerden.csv', low_memory=False)
print(f"Der Datensatz umfasst: {data.shape[0]} Zeilen und {data.shape[1]} Spalten")

In [ ]:
# Vorschau - Datensatz:
data.head()

In [ ]:
# Anzeige der Datensatz-Struktur:
data.info()

In [ ]:
# Anzeige von statistischen Kennzahlen:
data.describe()

In [ ]:
# Anzahl der undefinierten/nicht darstellbaren numerischen Werte (Not a Number, NaN) in jeder Spalte:
data.isnull().sum()

In [ ]:
# Überprüfung, ob die Spalte "Verbraucherbeschwerden" (consumer_complaint_narrative) im Datensatz existiert:
if 'consumer_complaint_narrative' in data.columns:
    print("Der Datensatz weist Kundenbeschwerden auf!")
    
# Zeilen-Filterung mit Beschwerden (ohne NaN-Werte):
    complaints = data[data['consumer_complaint_narrative'].notna()]['consumer_complaint_narrative']
    print("Es werden nur die Zeilen gefiltert, in denen ein Beschwerdetext vorhanden ist!")
    
# Gibt die Anzahl der Kundenbeschwerden aus oder einen Überprüfungshinweis bei keinem Fund:
    print(f"{len(complaints):,} Beschwerdetexte sind vorhanden von {len(data):,}".replace(",","."))
else:
    print("Die Kundenbeschwerden wurden nicht gefunden. Eine Überprüfung der Spaltennamen ist notwendig!")
    print(data.columns.tolist())

In [ ]:
# Textvorverarbeitung
def preprocess_text(text):
    # Überprüfung, ob der Text einen String enthält
    if not isinstance(text, str):
        return ""

    # Datensatz in Kleinbuchstaben umwandeln:
    text = text.lower()
    
    # Erntfernung von Sonderzeichen/Zahlen - nur Buchstaben und Leerzeichen bleiben erhalten:
    text = re.sub(r'[^a-z\s]', '', text)

    # Segmentierung des Datensatzes in Einheiten der Wortebene (Tokenisierung):
    tokens = word_tokenize(text, preserve_line=True)

    # Entfernung von Stoppwörtern:
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatisierung des Datensatzes:
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Mustererkennung und -entfernung:
    # re.fullmatch prüft die Wörter auf Muster:
    tokens = [word for word in tokens if not re.fullmatch(r'x{2,}', word)]

    # Wort-Entfernung, welche x-Sequenzen enthalten:
    tokens = [word for word in tokens if not re.search(r'x{2,}', word)]

    # Entfernen von kurzen Wörtern (Rauschen):
    tokens = [word for word in tokens if len(word) > 2]
    
    # Rückgabe als einzelner String mit Leerzeichen getrennt
    return ' '.join(tokens)
print("Alle Schritte wurden erfolgreich durchgeführt")

In [ ]:
# Rechtschreibkorrektur:
corrected_tokens = []
for word in tokens:
    if len(word) <=2:
        corrected_tokens.append(word)
        continue
        
# SymSpell für Rechtschreibkorrektur:        
    try:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            if suggestions[0].distance <= 1:
                corrected_tokens.append(suggestions[0].term)
            else:
                corrected_tokens.append(word)
        else:
            corrected_tokens.append(word)
    except:
        corrected_tokens.append(word)

In [ ]:
# Stichprobenartige Vorverarbeitung, um die Funktionalität zu überprüfen:
print("\nVorverarbeitungs-Test, um die Funktionalität zu überprüfen:")
sample = complaints.sample(5)
for i, text in enumerate(sample):
    print(f"Original {i+1}: {text[:100]}...")
    print(f"Verarbeitet {i+1}: {preprocess_text(text)[:100]}...")
    print("-" * 80)

In [ ]:
# Vorverarbeitung auf alle Texte anwenden:
processed_complaints = complaints.apply(preprocess_text)
print("Die Vorverarbeitung auf alle Texte ist abgeschlossen!")

In [ ]:
# Speicherung:
processed_df = pd.DataFrame({'processed_text': processed_complaints})
processed_df.to_csv('C:/Users/test/Desktop/Projekt_Data Analysis/Kundenbeschwerden_Vorverarbeitet.csv', index=False)
print("Der vorverarbeitete Datensatz wurde in einer CSV-Datei gespeichert!")